# Base de datos Cassandra

## Conexion y borrado de datos

In [8]:
%load_ext cql

The cql extension is already loaded. To reload it, use:
  %reload_ext cql


In [9]:
%%cql
DROP KEYSPACE practica_8anu;

'No results.'

In [10]:
%%cql
CREATE KEYSPACE practica_8anu 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [11]:
%cql USE practica_8anu;

'No results.'

## Creacion de tablas

** TABLA ESCALADORES **

In [12]:
%%cql 
CREATE TABLE escaladores (
    id_escalador    int,
    nombre          text,
    sexo            text,
    pais            text,
    anio_comienzo   int,
    PRIMARY KEY (id_escalador)
);

'No results.'

** TABLA DE ASCENSOS **

In [13]:
%%cql 
CREATE TABLE ascensos (
    uid                  uuid,
    id_escalador         int,
    nombre_escalador     text,
    pais_escalador       text,
    anio_comienzo        int,
    nombre_via           text,
    dificultad           int,
    grado_frances        text,
    tipo_encadenamiento  text,
    risco                text,
    pais_risco           text,
    PRIMARY KEY (id_escalador)
);

'No results.'

** TABLA DE ASCENSOS ACUMULADOS POR ESCALADOR **

In [14]:
%%cql 
CREATE TABLE ascensos_acumulados_hombres (
    id_escalador    int,
    num_ascensos    counter,
    dificultad_acum counter,
    PRIMARY KEY (id_escalador)
);

'No results.'

In [15]:
%%cql 
CREATE TABLE ascensos_acumulados_mujeres (
    id_escalador    int,
    num_ascensos    counter,
    dificultad_acum counter,
    PRIMARY KEY (id_escalador)
);

'No results.'

## Lectura de la informacion en Pandas

In [16]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = df_ascensos[['id_escalador','id_dificultad', 'tipo_encadenamiento', 'nombre_via', 'risco', 'sector', 'pais']];

## Desnormalización del data frame de ascensos

Voy a incluir la información del usuario en cada registro de ascenso, para ir creando los buckets que van interesando según las preguntas a resolver

In [17]:
df_ascensos = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'left');
df_ascensos = pd.merge(df_ascensos, df_dificultades, on = ['id_dificultad'], how = 'left');

# Renombrar campos por claridad
df_ascensos = df_ascensos.rename(
    columns={
        'id_dificultad' : 'dificultad',
        'nombre': 'nombre_escalador',
        'ciudad':'ciudad_escalador',
        'pais_y': 'pais_escalador',
        'comienzo': 'anio_comienzo',
        'pais_x': 'pais_risco'});


## Carga de datos en Cassandra

In [18]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('practica_8anu')

**CARGA EN TABLAS DE ASCENSOS Y ACUMULADOS**

In [19]:
def insert_ascensos(df):
    cql_insert_ascenso = """
        INSERT INTO ascensos (
            uid,
            id_escalador,
            nombre_escalador,
            pais_escalador,
            anio_comienzo,
            nombre_via,
            dificultad,
            grado_frances,
            tipo_encadenamiento,
            risco,
            pais_risco
        ) VALUES (now(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    

    
    for index in df.index:
        data = [
            df.ix[index,"id_escalador"],
            df.ix[index,"nombre_escalador"],
            df.ix[index,"pais_escalador"],
            df.ix[index,"anio_comienzo"],
            df.ix[index,"nombre_via"],
            df.ix[index,"dificultad"],
            df.ix[index,"grado_frances"],
            df.ix[index,"tipo_encadenamiento"],
            df.ix[index,"risco"],
            df.ix[index,"pais_risco"]
        ]

        session.execute(cql_insert_ascenso, data)

        if (df.ix[index,"sexo"] == 'Hombre'):
            session.execute("""UPDATE ascensos_acumulados_hombres
                               SET num_ascensos = num_ascensos + 1,
                                   dificultad_acum = dificultad_acum + %s
                               WHERE id_escalador = %s""",
                            [df.ix[index,"dificultad"], df.ix[index,"id_escalador"]])
        else:
            session.execute("""UPDATE ascensos_acumulados_mujeres
                               SET num_ascensos = num_ascensos + 1,
                                   dificultad_acum = dificultad_acum + %s
                               WHERE id_escalador = %s""",
                            [df.ix[index,"dificultad"], df.ix[index,"id_escalador"]])
        

In [20]:
insert_ascensos(df_ascensos)

** CARGA EN TABLA DE ESCALADORES **

In [21]:
def insert_escaladores(df):
    cql_insert = """
        INSERT INTO escaladores (
            id_escalador,
            nombre,
            sexo,
            pais,
            anio_comienzo
        ) VALUES (%s, %s, %s, %s, %s)"""
    
    for index in df.index:
        data = [
            df.ix[index, "id_escalador"],
            df.ix[index, "nombre"],
            df.ix[index, "sexo"],
            df.ix[index, "pais"],
            df.ix[index, "comienzo"]]
        session.execute(cql_insert, data)

In [22]:
insert_escaladores(df_escaladores)

# RESPUESTAS A LAS PREGUNTAS

In [28]:
def execute_query(cql, data=[]):
    rows = session.execute(cql, data)
    return pd.DataFrame(list(rows))

### 1.a) Los 10 escaladores (hombres) más activos (orden auxiliar por Id)

Esto requiere varias consultas a dos tablas diferentes:
- Una consulta a la tabla de acumulados de hombres de la que se obtienen los ids de los escaladores con mas ascensos acumulados
- 10 consultas a la tabla de escaladores, para sacar los datos del escalador.

Por desgracia las tablas con columnas counter:
- no pueden contener columnas de otro tipo (excepto una unica columna que hace de partition key): https://stackoverflow.com/questions/32562500/cassandra-non-counter-family
- no pueden poner como columna de ordenamiento una clustering key, con lo que no se puede meter num_ascensos como clustering key y solicitar ordenacion por la misma. Esto hace que la ordenación haya hacerla fuera.


In [45]:
cql = """
SELECT *
FROM ascensos_acumulados_hombres
"""
df_id_acum = execute_query(cql)

df_id_acum = df_id_acum_hombres.sort_values(['num_ascensos', 'id_escalador'],ascending=[False, False]).head(10)

In [46]:
cql = """
SELECT *
FROM escaladores
WHERE id_escalador=%s
"""

df = pd.DataFrame()
for index in df_id_acum.index:
    df_escalador = execute_query(cql, [df_id_acum.ix[index,'id_escalador']])
    df_escalador["num_ascensos"] = df_id_acum.ix[index, "num_ascensos"]
    
    df = df.append(df_escalador)
    
columnas = ["id_escalador", "nombre", "sexo", "pais", "num_ascensos"]
df[columnas]  
    

,id_escalador,nombre,sexo,pais,num_ascensos
0,50884,Christopher Leonetti,Hombre,USA,47
0,20384,Clemens Kurth,Hombre,DEU,26
0,20095,Matthias Schuster,Hombre,DEU,26
0,66250,Kuba Kaminski,Hombre,POL,24
0,9171,Laurenz Trawnicek,Hombre,AUT,23
0,46622,Raúl Crespo,Hombre,ESP,19
0,42086,Marcin Opozda,Hombre,POL,19
0,66466,Christian Boehme,Hombre,DEU,18
0,47732,Thomas de Fleurian,Hombre,FRA,18
0,35847,Zack Bum,Hombre,DEU,18


### 1.b) Los 10 escaladoras (mujeres) más activas (orden auxiliar por Id)

In [47]:
cql = """
SELECT *
FROM ascensos_acumulados_mujeres
"""
df_id_acum = execute_query(cql)

df_id_acum = df_id_acum_hombres.sort_values(['num_ascensos', 'id_escalador'],ascending=[False, False]).head(10)

In [48]:
cql = """
SELECT *
FROM escaladores
WHERE id_escalador=%s
"""

df = pd.DataFrame()
for index in df_id_acum.index:
    df_escalador = execute_query(cql, [df_id_acum.ix[index,'id_escalador']])
    df_escalador["num_ascensos"] = df_id_acum.ix[index, "num_ascensos"]
    
    df = df.append(df_escalador)
    
columnas = ["id_escalador", "nombre", "sexo", "pais", "num_ascensos"]
df[columnas] 

,id_escalador,nombre,sexo,pais,num_ascensos
0,50884,Christopher Leonetti,Hombre,USA,47
0,20384,Clemens Kurth,Hombre,DEU,26
0,20095,Matthias Schuster,Hombre,DEU,26
0,66250,Kuba Kaminski,Hombre,POL,24
0,9171,Laurenz Trawnicek,Hombre,AUT,23
0,46622,Raúl Crespo,Hombre,ESP,19
0,42086,Marcin Opozda,Hombre,POL,19
0,66466,Christian Boehme,Hombre,DEU,18
0,47732,Thomas de Fleurian,Hombre,FRA,18
0,35847,Zack Bum,Hombre,DEU,18


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de via ascendete)

### 3. Dificultad media de los ascensos del escalador más activo

### 4.a) Los 10 ascensos mas dificiles

### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente